In [12]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from glob import glob
from utils import iter_df_read

BASE = "/media/giani/Gianicosas/Magister/Proyecto/"

In [16]:
df = pd.read_csv(f"{BASE}work_data/matriz_desercion_4_dummy_con_id.csv.gz")
df.head()

,mrun,codigo_unico,dur_estudio_carr,dur_proceso_tit,desertor_1,prom_notas_media,fscu,gratuidad,beca,dif_ing_ult_mat,...,tipo_ensenanza_colegio_tp_industrial,tipo_ensenanza_colegio_tp_tecnica,dependencia_colegio_municipal,dependencia_colegio_otro,dependencia_colegio_particular_pagado,dependencia_colegio_particular_subvencionado,area_conoc_inst_adm_hum,area_conoc_inst_ciencias_tec,area_conoc_inst_no_ip,area_conoc_inst_sin_tendencia
0,25866904.0,I426S1C7J2V2,4,1,1,6.4,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,25768831.0,I143S26C376J2V1,5,1,1,5.5,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
2,25768815.0,I273S1C5J2V1,6,1,0,4.7,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,25768749.0,I143S29C26J2V1,5,1,0,6.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,25768671.0,I143S15C30J1V1,5,1,0,6.4,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [1]:
shap_best = [
    "dif_ing_ult_mat", "valor_arancel", "beca", "prom_notas_media", "gen_alu_hombre",
    "edad", "dur_proceso_tit", "acreditada_carr_acreditada", "dur_estudio_carr",
    "jornada_vespertino", "dependencia_colegio_municipal", "jornada_diurno",
    "area_conocimiento_tecnologia", "gratuidad", "area_conoc_inst_sin_tendencia",
    "region_sede_metropolitana", "acreditada_inst_acreditada",
    "area_conocimiento_arte_y_arquitectura", "es_cft_estatal", "tipo_ensenanza_colegio_hc"
]

In [3]:
len(['dur_estudio_carr', 'dur_proceso_tit', 'prom_notas_media', 'fscu',
       'gratuidad', 'beca', 'dif_ing_ult_mat', 'edad', 'valor_arancel',
       'misma_region', 'es_cft_estatal', 'gen_alu_hombre',
       'gen_alu_mujer', 'tipo_inst_1_centros_de_formacion_tecnica',
       'tipo_inst_1_institutos_profesionales', 'jornada_a_distancia',
       'jornada_diurno', 'jornada_otro', 'jornada_semipresencial',
       'jornada_vespertino', 'region_sede_antofagasta',
       'region_sede_arica_y_parinacota', 'region_sede_atacama',
       'region_sede_biobio', 'region_sede_coquimbo',
       'region_sede_la_araucania', 'region_sede_lib_gral_b_ohiggins',
       'region_sede_los_lagos', 'region_sede_los_rios',
       'region_sede_magallanes', 'region_sede_maule',
       'region_sede_metropolitana', 'region_sede_valparaiso',
       'region_sede_nuble', 'area_conocimiento_administracion_y_comercio',
       'area_conocimiento_agropecuaria',
       'area_conocimiento_arte_y_arquitectura',
       'area_conocimiento_ciencias_basicas',
       'area_conocimiento_ciencias_sociales',
       'area_conocimiento_educacion', 'area_conocimiento_humanidades',
       'area_conocimiento_salud', 'area_conocimiento_tecnologia',
       'acreditada_carr_acreditada', 'acreditada_carr_no_acreditada',
       'acreditada_inst_acreditada', 'acreditada_inst_no_acreditada',
       'tipo_ensenanza_colegio_hc', 'tipo_ensenanza_colegio_otro',
       'tipo_ensenanza_colegio_tp_comercial',
       'tipo_ensenanza_colegio_tp_industrial',
       'tipo_ensenanza_colegio_tp_tecnica',
       'dependencia_colegio_municipal', 'dependencia_colegio_otro',
       'dependencia_colegio_particular_pagado',
       'dependencia_colegio_particular_subvencionado',
       'area_conoc_inst_ciencias_tec', 'area_conoc_inst_no_ip',
       'area_conoc_inst_sin_tendencia'])

59

# Matriculas 2015

In [14]:
cols_mat = [
    'cat_periodo', 'mrun', "codigo_unico", "anio_ing_carr_ori", "tipo_inst_1"
]

instituciones = ['Institutos Profesionales', 'Centros de Formación Técnica']

matriculados = pd.concat([
    iter_df_read(f, "tipo_inst_1", instituciones, sep=";", usecols=cols_mat)
    for f in tqdm(
        glob(f"{BASE}/raw_data/post_2015/20220719_Matrícula_Ed_Superior_*.csv"),
        total=8
    )
]).dropna(subset="mrun").drop_duplicates().reset_index(drop=True)

matriculados = matriculados[
    (matriculados["anio_ing_carr_ori"] == 2015)
]

print(f"Cantidad de registros: {matriculados.shape[0]}\n")
print(f"Cantidad de nulos: \n{matriculados.isna().sum()}")
matriculados.head()

  0%|          | 0/8 [00:00<?, ?it/s]

Cantidad de registros: 491221

Cantidad de nulos: 
cat_periodo          0
codigo_unico         0
mrun                 0
anio_ing_carr_ori    0
tipo_inst_1          0
dtype: int64


,cat_periodo,codigo_unico,mrun,anio_ing_carr_ori,tipo_inst_1
0,2015,I374S1C9J1V1,5.0,2015,Centros de Formación Técnica
1,2015,I498S6C132J2V1,37.0,2015,Centros de Formación Técnica
3,2015,I111S14C309J2V1,118.0,2015,Institutos Profesionales
6,2015,I106S3C64J4V1,253.0,2015,Institutos Profesionales
8,2015,I221S1C57J1V1,370.0,2015,Centros de Formación Técnica


In [20]:
df_chico = df[shap_best + ["desertor_1", "mrun", "codigo_unico"]]

In [31]:
df_left = pd.merge(
    df_chico,
    matriculados[["codigo_unico", "mrun"]].drop_duplicates(),
    how="left"
)

df_chico.shape, matriculados.shape, df_left.shape

((885579, 23), (491221, 5), (885579, 23))

In [32]:
df_left["desertor_1"].value_counts()

0    672747
1    212832
Name: desertor_1, dtype: int64

In [33]:
df_left.drop(columns=["mrun", "codigo_unico"]).to_csv(
    f"{BASE}work_data/matriz_desercion_4_mats_2015_dep.csv.gz",
    index=False,
    compression="gzip"
)